## Visualising interactive maps
This notebook is a simple guide on how to visualize raster images using the **leafmap** and **localtileserver** libraries in combination with **jupyter-server-proxy**.

Using these packages locally is quite easy; however, in a JupyterHub environment, raster visualisation libraries typically need **jupyter-server-proxy** to function.

To help users configure these packages using environment variables, this notebook was created.

First, we start by retrieving the data that we use in this example. We will demonstrate the data visualisations with a DEM geotiff.

In [19]:
import pystac_client

URL = "https://stac.dataspace.copernicus.eu/v1"
cat = pystac_client.Client.open(URL)
cat.add_conforms_to("ITEM_SEARCH")
item = list(cat.get_items("Copernicus_DSM_COG_30_N49_00_E007_00_DEM"))[0]
s3_path = item.assets["data"].href.replace("s3://eodata/", "")
s3_path

'auxdata/CopDEM_COG/copernicus-dem-90m/Copernicus_DSM_COG_30_N49_00_E007_00_DEM/Copernicus_DSM_COG_30_N49_00_E007_00_DEM.tif'

Now lets download the file!

In [20]:
import boto3
import os

access_key = os.environ.get("CDSE_S3_ACCESS_KEY")
secret_key = os.environ.get("CDSE_S3_SECRET_KEY")

session = boto3.session.Session()
s3 = boto3.resource(
    "s3",
    endpoint_url="https://eodata.dataspace.copernicus.eu",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name="default",
)  # generated secrets


def download(bucket, product: str, target: str = "") -> str:
    """
    Downloads every file in bucket with provided product as prefix

    Raises FileNotFoundError if the product was not found

    Args:
        bucket: boto3 Resource bucket object
        product: Path to product
        target: Local catalog for downloaded files. Should end with an `/`. Default current directory.
    """
    files = bucket.objects.filter(Prefix=product)

    if not list(files):
        raise FileNotFoundError(f"Could not find any files for {product}")
    for file in files:
        os.makedirs(os.path.dirname(file.key), exist_ok=True)
        if not os.path.isdir(file.key):
            bucket.download_file(file.key, f"{target}{file.key}")
    return f"{target}{file.key}"


# path to the product to download
file_path = download(s3.Bucket("eodata"), s3_path)
file_path

'auxdata/CopDEM_COG/copernicus-dem-90m/Copernicus_DSM_COG_30_N49_00_E007_00_DEM/Copernicus_DSM_COG_30_N49_00_E007_00_DEM.tif'

### Localtileserver
The localtileserver allows users to create a backend tileserver that converts local raster data (e.g. GeoTIFFs) into web map tiles (XYZ tiles) so they can be displayed interactively in a web map or notebook. We can use the tileserver in combination with the ipyleaflet library to visualise rasters. Here is an example on how to run this package.

In [21]:
from localtileserver import get_leaflet_tile_layer, TileClient
from ipyleaflet import Map, FullScreenControl
import os

# Update LOCALTILESERVER_CLIENT_PREFIX to handle the JUPYTERHUB_SERVICE_PREFIX.
if (
    os.environ.get("JUPYTERHUB_SERVICE_PREFIX")
    and "LOCALTILESERVER_CLIENT_PREFIX" not in os.environ
):
    os.environ[
        "LOCALTILESERVER_CLIENT_PREFIX"
    ] = f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}proxy/{{port}}"

# After which, you can create a tile server from local raster file
tile_client = TileClient(file_path)

# Create the layers using the tileservers
layer = get_leaflet_tile_layer(tile_client)

# Create the map and add the layers
m = Map(center=tile_client.center(), zoom=8)
m.add_layer(layer)
m.add_control(FullScreenControl())

# Visualise
m

Map(center=[49.500417, 7.499583], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

### Leafmap
Similarly to localtileserver, Leafmap is a package used to create interactive maps with minimal coding. It is built on several open-source libraries such as folium and ipyleaflet, which make it easy to generate interactive visualisations. Below is a code snippet that demonstrates how to visualize a raster.

Leafmap appends an extra '/' to the end of the JUPYTERHUB_SERVICE_PREFIX. However, by default, JUPYTERHUB_SERVICE_PREFIX already ends with a '/'. This duplication can cause rendering issues, which can be resolved by **temporarily** removing the trailing '/' from the prefix.

In [22]:
import os
import leafmap.leafmap as leafmap

if os.environ.get("JUPYTERHUB_SERVICE_PREFIX"):
    os.environ["JUPYTERHUB_SERVICE_PREFIX"] = (
        os.environ["JUPYTERHUB_SERVICE_PREFIX"][:-1]
        if os.environ["JUPYTERHUB_SERVICE_PREFIX"][-1] == "/"
        else os.environ["JUPYTERHUB_SERVICE_PREFIX"]
    )

m = leafmap.Map()
m.add_raster(file_path)
m

Map(center=[49.500417, 7.499583], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

When you are done with your render, add the '/' back at the end of the prefix.

In [23]:
if os.environ.get("JUPYTERHUB_SERVICE_PREFIX"):
    os.environ["JUPYTERHUB_SERVICE_PREFIX"] = (
        os.environ["JUPYTERHUB_SERVICE_PREFIX"]
        if os.environ["JUPYTERHUB_SERVICE_PREFIX"][-1] == "/"
        else f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}/"
    )